In [13]:
import glob
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
import skimage.transform as skt
import skimage
import numpy as np
import ipywidgets as widgets

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord 
from astropy.visualization import ZScaleInterval

from photutils.aperture import CircularAperture
from RetrieveSource import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact


In [10]:
############
# Get File #
############

def get_file(id,segment=1,extension=1,filter='um2',galaxy='SMC'):
    file_location = f"/media/bethany/Seagate Portable Drive/SwiftData/Astrometry/{galaxy}"
    labels = ['RAhr','DEdeg','Umag','e_Umag','Bmag','e_Bmag','Vmag','e_Vmag','Imag','e_Imag','Flag','Jmag','e_Jmag','Hmag','e_Hmag','Ksmag','e_Ksmag']
    fname = glob.glob(f"{file_location}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.new")
    cname = glob.glob(f"{file_location}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.full.dat")
    HDR = fits.open(fname[0])
    CAT = pd.read_csv(cname[0],delimiter='\s+',names=labels)
    return HDR, CAT, cname

def plot_file(id,segment=1,extension=1,filter='um2',xdim=None,ydim=None):   
    Z = ZScaleInterval()
    HDR,CAT,cname = get_file(id,segment=1,extension=1,filter='um2')
    data = HDR[0].data
    if xdim == None:
        vmin,vmax = Z.get_limits(data)
    else:
        vmin,vmax = Z.get_limits(data[ydim[0]:ydim[1],xdim[0]:xdim[1]])
    f= plt.figure(figsize=(10,10))
    plt.imshow(data,origin='lower',vmin=vmin,vmax=vmax)
    plt.xlim(xdim)
    plt.ylim(ydim)
    
def get_pixel_coordinates(HDR,CAT):
    co = SkyCoord(CAT['RAhr'],CAT['DEdeg'],unit=(u.hourangle,u.deg))
    x,y = co.to_pixel(WCS(HDR[0].header))
    CAT['X'] = x
    CAT['Y'] = y
    return x,y 

def reformat_pixel_coordinates(x,y):
    # If you need coordinates as integers or need to move from edges at all
    x_co = np.array([int(np.round(x_)) for x_ in X[Y<1312]])
    y_co = np.array([int(np.round(y_)) for y_ in Y[Y<1312]])
    return x_co,y_co

def get_data(id) :
    HDR,CAT,cname = get_file(id,segment=1,extension=1,filter='um2')
    return HDR[0].data

def sane_plot(data) : 
    f= plt.figure(figsize=(5,5))
    plt.imshow(data, origin='lower', vmin=0, vmax=15);

In [82]:
def d(x0,y0,x,y):
    distance = np.sqrt((x-x0)**2 + (y-y0)**2)
    distance = np.delete(distance,np.where(distance == 0.))
    return distance
def check(min_x,min_y):
    for x,y in zip(min_x,min_y):
        dist = d(x,y,min_x,min_y)
        if np.min(dist) < 1:
            print('warning')

def check_section(n_x,n_y,obsid=40416):
    
    data = get_data(obsid);
    data = skt.rotate(skimage.util.img_as_float64(data),24)
    data = data[n_y:n_y+50,n_x:n_x+50];

    if len(data[data!=0]) == 0: 
        
        print("No Data in Section")
    else:
    
        fourier_data = np.fft.fftshift(np.fft.fft2(data));

        min_y,min_x = np.where(np.isclose(np.log(abs(fourier_data)), np.min(np.log(abs(fourier_data))),atol=1))

        Z = ZScaleInterval()
        vmin,vmax = Z.get_limits(data)
        vmin1,vmax1 = Z.get_limits(np.log(abs(fourier_data)))

        f, axes = plt.subplots(1,2,figsize=(10,5))

        im1 = axes[0].imshow(data,origin="lower",vmin=vmin,vmax=vmax)
        im2 = axes[1].imshow(np.log(abs(fourier_data)),origin="lower",vmin=vmin1,vmax=vmax1)

        axes[1].scatter(min_x,min_y,c='r')
        axes[0].set_title("Original Section")
        axes[1].set_title("Fourier Transform")
        
        divider = make_axes_locatable(axes[0])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im1, cax=cax)
    
        divider = make_axes_locatable(axes[1])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im2, cax=cax)
    
        check(min_x,min_y)
            
interact(check_section,
         n_x=widgets.IntSlider(min=139, max=1303, step=1, value=0),
         n_y=widgets.IntSlider(min=164, max=1316, step=1, value=0),
         obsid=widgets.IntSlider(min=40415, max=40465, step=1, value=40416))

interactive(children=(IntSlider(value=139, description='n_x', max=1303, min=139), IntSlider(value=164, descrip…

<function __main__.check_section(n_x, n_y, obsid=40416)>